In [1]:
import pandas as pd
import numpy as np
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import time
from selenium.common.exceptions import NoSuchElementException,ElementNotInteractableException,StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
import re

### Q1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars

In [2]:
driver=webdriver.Edge(r'msedgedriver.exe')
driver.get('https://www.amazon.in')
search=input()
driver.find_element_by_xpath("//input[@id='twotabsearchtextbox']").send_keys(search)
driver.find_element_by_xpath('//span[@id="nav-search-submit-text"]').click()

guitar


### Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.


In [3]:
url=[]
brand=[]
name=[]
rating=[]
no_rating=[]
price=[]
exc=[]
exp_delivery=[]
avail=[]
other_details=[]

#Scraping 3 pages
for i in range(3):
    
    #URL
    for j in driver.find_elements_by_xpath('//h2[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-4"]/a'):
        url.append(j.get_attribute('href'))

    #Clicking on Next Button
    try:
        delay=10
        WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CLASS_NAME,'a-last')))
        driver.find_element_by_xpath('//li[@class="a-last"]/a').click()
    except TimeoutException as e:
        break
    time.sleep(3)
        
                
#For every URL
for u in url:
    driver.get(u)
        
    #Brand   
    try:
        table=driver.find_elements_by_xpath('//table[@class="a-normal a-spacing-micro"]/tbody/tr/td')
        found=0
        for index,element in enumerate(table):
            if element.text=='Brand':
                found=1
                brand.append(table[index+1].text)
                break
        if found==0:
            brand.append('-')
    except NoSuchElementException as e:
        try:
            table1=driver.find_elements_by_xpath('//table[@id="productDetails_techSpec_section_1"]/tbody/tr')
            found=0
            for i in table1:
                if i.text.split()[0]=='Brand':
                    found=1
                    l=i.text.split()[1:]
                    s=''
                    for words in l:
                        s=s+' '+words
                    brand.append(s)
                    break
            if found==0:
                brand.append('-')
        except NoSuchElementException as e:
            brand.append('-')
    
    #Name
    name.append(driver.find_element_by_xpath('//span[@id="productTitle"]').text)
        
    #Ratings
    try:
        rating.append(driver.find_element_by_xpath('//span[@class="a-size-base a-nowrap"]/span').text)
    except NoSuchElementException as e:
        rating.append('-')
        
    #Number of ratings
    try:
        no_rating.append(driver.find_element_by_xpath('//div[@class="a-row a-spacing-medium averageStarRatingNumerical"]/span').text)
    except NoSuchElementException as e:
        no_rating.append('-')
            
    #price
    try:
        price.append(driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]').text)
    except NoSuchElementException as e:
        try:
            price.append(driver.find_element_by_xpath('//span[@id="priceblock_dealprice"]').text)
        except NoSuchElementException as e:
            try:
                price.append(driver.find_element_by_xpath('//span[@id="priceblock_saleprice"]').text)
            except NoSuchElementException as e:
                price.append('-')
            
    #Return/Exchange
    try:
        exc.append(driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span').text)
    except NoSuchElementException as e:
        exc.append('-')
        
    #Expected Delivery
    try:
        exp_delivery.append(driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b').text)
    except NoSuchElementException as e:
        try:
            if driver.find_element_by_xpath('//*[@id="outOfStock"]')!='':
                exp_delivery.append('Out of Stock')
        except NoSuchElementException as e:
            exp_delivery.append('-')
        
    #Availibility
    try:
        avail.append(driver.find_element_by_xpath('//div[@id="availability"]/span').text)
    except NoSuchElementException as e:
        avail.append('Out of Stock')
        
    #Other Details
    try:
        other_details.append(driver.find_element_by_xpath('//div[@id="productDetails_db_sections"]').text)
    except NoSuchElementException as e:
        try:
            other_details.append(driver.find_element_by_xpath('//div[@id="detailBullets_feature_div"]').text)
        except NoSuchElementException as e:
            other_details.append('-')
        
            
data=pd.DataFrame()
data['Brand']=brand
data['Name']=name
data['Ratings']=rating
data['Number of Ratings']=no_rating
data['Price']=price
data['Return/Exchange']=exc
data['Expected Delivery']=exp_delivery
data['Availability']=avail
data['Other Details']=other_details
data['URL']=url

In [4]:
data

,Brand,Name,Ratings,Number of Ratings,Price,Return/Exchange,Expected Delivery,Availability,Other Details,URL
0,Kadence,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,3.9 out of 5,860 global ratings,"₹ 6,499.00",7 Days Replacement,"Wednesday, July 7",In stock.,Additional Information\nASIN B01HRIHJWY\nCusto...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Kadence,"Kadence Frontier Series,Blue Acoustic Guitar W...",4 out of 5,249 global ratings,"₹ 4,999.00",7 Days Replacement,"Wednesday, July 7",In stock.,Additional Information\nASIN B01CJJGEVS\nCusto...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Kadence,Kadence Slowhand Premium Jumbo Semi Acoustic G...,4.2 out of 5,223 global ratings,"₹ 10,499.00",7 Days Replacement,"Wednesday, July 7",In stock.,Additional Information\nASIN B077SZ667X\nCusto...,https://www.amazon.in/gp/slredirect/picassoRed...
3,Guitar Bro,GUITAR BRO - COMBO (Blue Acoustic Guitar for B...,4.1 out of 5,23 global ratings,"₹ 6,499.00",7 Days Replacement,July 8 - 12,In stock.,Additional Information\nASIN B07TC7TWCR\nCusto...,https://www.amazon.in/gp/slredirect/picassoRed...
4,Juârez,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",3.9 out of 5,"10,671 global ratings","₹ 2,499.00",7 Days Replacement,"Tuesday, July 6",In stock.,Additional Information\nASIN B017NPCSLI\nCusto...,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
153,Cort,Cort AD810 Dreadnought Acoustic Guitar with Gi...,4.3 out of 5,113 global ratings,-,7 Days Replacement,-,,Additional Information\nASIN B07KMBVR6V\nCusto...,https://www.amazon.in/Cort-AD810-Dreadnought-A...
154,Kadence,"Kadence AstroMan Electric Guitar, 21 FRETS, RO...",4 out of 5,31 global ratings,"₹ 6,499.00",7 Days Replacement,"Wednesday, July 7",In stock.,Additional Information\nASIN B07BGXN7P6\nCusto...,https://www.amazon.in/Kadence-Electric-Guitar-...
155,Asago,Asago 4 String Children's Musical Guitar Instr...,2.6 out of 5,3 global ratings,₹ 799.00,7 Days Replacement,July 7 - 8,In stock.,Additional Information\nASIN B08VD8JLHT\nCusto...,https://www.amazon.in/Asago-Childrens-Instrume...
156,RockJam,RockJam RJEG02-SK-BB Electric Guitar Starter K...,4.1 out of 5,343 global ratings,"₹ 7,949.00",7 Days Replacement,"Tuesday, July 6",In stock.,Additional Information\nASIN B0747TD557\nCusto...,https://www.amazon.in/RockJam-RJEG02-SK-BB-Ele...


### Q3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [5]:
def download_images(picture):
    driver=webdriver.Edge(r'msedgedriver.exe')
    driver.get('https://images.google.com')

    box=driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
    box.send_keys(picture)
    box.send_keys(Keys.ENTER)

    for i in range(5):
        driver.execute_script("window.scrollBy(0,10000)")
        time.sleep(2)
        try:
            driver.find_element_by_xpath('//input[@value="Show more results"]').click()
        except ElementNotInteractableException:
            pass

    image_count=0
    for img in driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]'):
        img.screenshot('C:\\Users\\aush1\\OneDrive\\Documents\\Datatrained\\Internship\\Project4 Selenium\\images\\'+picture+str(image_count)+'.png')
        image_count+=1
        if image_count==100:
            break


In [6]:
download_images('fruits')

In [7]:
download_images('cars')

In [8]:
download_images('Machine Learning')

### Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. In case if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [9]:
def mobileInfo(phone_name,url):
    driver=webdriver.Edge(r'msedgedriver.exe')
    driver.get(url)

    #Closing the login prompt
    try:
        driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]').click()
    except NoSuchElementException:
        pass

    #Typing phone name and clicking search button
    driver.find_element_by_xpath('//div[@class="_3OO5Xc"]/input').send_keys(phone_name)
    driver.find_element_by_xpath('//button[@class="L0Z3Pu"]').click()

    #Initializing list for all the information to be extracted
    url=[]
    brand=[]
    name=[]
    colour=[]
    ram=[]
    storage=[]
    pcamera=[]
    scamera=[]
    dispsize=[]
    dispreso=[]
    processor=[]
    processor_cores=[]
    batt_capacity=[]
    price=[]

    #Extracting URL of all the phones
    delay=10
    WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CLASS_NAME,'_1fQZEK')))
    for i in driver.find_elements_by_xpath('//a[@class="_1fQZEK"]'):
        url.append(i.get_attribute('href'))

    #Iterating through the URL list and extracting all phone's info
    for i in url:
        driver.get(i)
        time.sleep(2)

        #Flag variables
        n=0
        c=0
        r=0
        s=0
        pca=0
        sc=0
        ds=0
        dr=0
        pt=0
        pc=0
        bc=0
        
        #Extracting phone Brand
        brand.append(driver.find_element_by_xpath('//h1[@class="yhB1nd"]/span').text.split()[0])

        #Extracting phone Price
        price.append(driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]').text)

        #Clicking on the readmore label
        driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
        time.sleep(2)

        #Extracting all the specification's table rows
        specs=driver.find_elements_by_xpath('//tr[@class="_1s_Smc row"]')

        for row in specs:
            label=row.find_element_by_xpath('td[1]').text

            #Extracting phone Name
            if label=='Model Name':
                name.append(row.find_element_by_xpath('td[2]').text)
                n=1

            #Extracting phone Color
            if label=='Color':
                colour.append(row.find_element_by_xpath('td[2]').text)
                c=1

            ##Extracting phone RAM
            if label=='RAM':
                ram.append(row.find_element_by_xpath('td[2]').text)
                r=1

            #Extracting phone Storage
            if label=='Internal Storage':
                storage.append(row.find_element_by_xpath('td[2]').text)
                s=1

            #Extracting phone Primary Camera
            if label=='Primary Camera':
                pcamera.append(row.find_element_by_xpath('td[2]').text)
                pca=1

            #Extracting phone Secondary Camera
            if label=='Secondary Camera':
                scamera.append(row.find_element_by_xpath('td[2]').text)
                sc=1

            #Extracting phone Display Size
            if label=='Display Size':
                dispsize.append(row.find_element_by_xpath('td[2]').text)
                ds=1

            #Extracting phone Resolution
            if label=='Resolution':
                dispreso.append(row.find_element_by_xpath('td[2]').text)
                dr=1

            #Extracting phone Processor
            if label=='Processor Type':
                processor.append(row.find_element_by_xpath('td[2]').text)
                pt=1

            #Extracting phone Processor Core
            if label=='Processor Core':
                processor_cores.append(row.find_element_by_xpath('td[2]').text)
                pc=1

            #Extracting phone Battery Capacity
            if label=='Battery Capacity':
                batt_capacity.append(row.find_element_by_xpath('td[2]').text)
                bc=1

        #Checking Flag variables
        if n==0:
            name.append('-')
        if c==0:
            colour.append('-')
        if r==0:
            ram.append('-')
        if s==0:
            storage.append('-')
        if pca==0:
            pcamera.append('-')
        if sc==0:
            scamera.append('-')
        if ds==0:
            dispsize.append('-')
        if dr==0:
            dispreso.append('-')
        if pt==0:
            processor.append('-')
        if pc==0:
            processor_cores.append('-')
        if bc==0:
            batt_capacity.append('-')

    #Creating DataFrame
    data=pd.DataFrame()
    data['Name']=name
    data['Brand']=brand
    data['Colour']=colour
    data['Ram']=ram
    data['Storage']=storage
    data['Primary Camera']=pcamera
    data['Secondary Camera']=scamera
    data['Display Size']=dispsize
    data['Display Resolution']=dispreso
    data['Processor']=processor
    data['Processor Cores']=processor_cores
    data['Battery Capacity']=batt_capacity
    data['Price']=price
    data['URL']=url
    return data

In [10]:
mobile=input('Which phone you want the information of?')
df=mobileInfo(mobile,'https://www.flipkart.com/')
df

Which phone you want the information of?samsung s20


,Name,Brand,Colour,Ram,Storage,Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price,URL
0,Galaxy S20,SAMSUNG,Cosmic Gray,8 GB,128 GB,64MP + 12MP + 12MP,10MP Front Camera,15.75 cm (6.2 inch),3200 x 1440 Pixels,Exynos 990,Octa Core,4000 mAh,"₹52,905",https://www.flipkart.com/samsung-galaxy-s20-co...
1,Galaxy S20,SAMSUNG,Cloud Blue,8 GB,128 GB,64MP + 12MP + 12MP,10MP Front Camera,15.75 cm (6.2 inch),3200 x 1440 Pixels,Exynos 990,Octa Core,4000 mAh,"₹49,980",https://www.flipkart.com/samsung-galaxy-s20-cl...
2,Galaxy S20,SAMSUNG,Cloud Pink,8 GB,128 GB,64MP + 12MP + 12MP,10MP Front Camera,15.75 cm (6.2 inch),3200 x 1440 Pixels,Exynos 990,Octa Core,4000 mAh,"₹74,000",https://www.flipkart.com/samsung-galaxy-s20-cl...
3,Galaxy S20 FE,SAMSUNG,Cloud Navy,8 GB,128 GB,12MP + 12MP + 8MP Rear Camera,32MP Front Camera,16.51 cm (6.5 inch),2400 x 1080 Pixels,Exynos Octa Core,Octa Core,4500 mAh,"₹37,999",https://www.flipkart.com/samsung-galaxy-s20-fe...
4,Galaxy S20 FE,SAMSUNG,Cloud White,8 GB,128 GB,12MP + 12MP + 8MP Rear Camera,32MP Front Camera,16.51 cm (6.5 inch),2400 x 1080 Pixels,Exynos Octa Core,Octa Core,4500 mAh,"₹37,999",https://www.flipkart.com/samsung-galaxy-s20-fe...
5,Galaxy S20+,SAMSUNG,Cosmic Gray,8 GB,128 GB,64 MP + 12 MP +12 MP + VGA Depth Camera,10MP Front Camera,17.02 cm (6.7 inch),3200 x 1440 Pixels,Exynos 990,Octa Core,4500 mAh,"₹54,999",https://www.flipkart.com/samsung-galaxy-s20-co...
6,Galaxy S20 Ultra,SAMSUNG,Cosmic Gray,12 GB,128 GB,108 MP + 48 MP + 12 MP + VGA Depth Camera,40MP Front Camera,17.53 cm (6.9 inch),3200 x 1440 Pixels,Exynos 990,Octa Core,5000 mAh,"₹1,03,000",https://www.flipkart.com/samsung-galaxy-s20-ul...
7,Galaxy A20s,SAMSUNG,Black,4 GB,64 GB,13MP + 8MP + 5MP,8MP Front Camera,16.51 cm (6.5 inch),720 x 1520 Pixels,Qualcomm SDM450-B01,Octa Core,4000 mAh,"₹14,900",https://www.flipkart.com/samsung-galaxy-a20s-b...
8,Galaxy S20 FE,SAMSUNG,Cloud Mint,8 GB,128 GB,12MP + 12MP + 8MP Rear Camera,32MP Front Camera,16.51 cm (6.5 inch),2400 x 1080 Pixels,Exynos Octa Core,Octa Core,4500 mAh,"₹40,000",https://www.flipkart.com/samsung-galaxy-s20-fe...
9,Galaxy A20s,SAMSUNG,Green,3 GB,32 GB,13MP + 8MP + 5MP,8MP Front Camera,16.51 cm (6.5 inch),720 x 1520 Pixels,Qualcomm SDM450-B01,Octa Core,4000 mAh,"₹12,899",https://www.flipkart.com/samsung-galaxy-a20s-g...


### Q5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps

In [11]:
def coordinates(place):
    driver=webdriver.Edge(r'msedgedriver.exe')
    driver.get('https://www.google.co.in/maps')
    
    search=driver.find_element_by_xpath('//input[@id="searchboxinput"]')
    search.send_keys(place) #typing place name on the searchbox
    search.send_keys(Keys.ENTER) #pressing enter in the search box
    time.sleep(3)
    
    #Getting the url of the page
    url_string = driver.current_url
    
    #Extracting the latitudes and longitudes from the url
    lat_long = re.findall('@(.*),',url_string)
    lat_long=lat_long[0].split(',')
    print('Latitude : {}'.format(lat_long[0]))
    print('Longitude : {}'.format(lat_long[1]))

In [12]:
coordinates('Dehradun')

Latitude : 30.3256502
Longitude : 77.8770476


### Q6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [13]:
def funding_details(url):
    driver=webdriver.Edge(r'msedgedriver.exe')
    driver.get(url) #loading url
    driver.maximize_window() #maximizing window

    date=[]
    startup=[]
    industry=[]
    sub_industry=[]
    location=[]
    investor=[]
    inv_type=[]
    amount=[]
    
    #Clicking on funding details tab
    driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').click()
    time.sleep(3)
    
    #Iterating through months
    for month in range(48,51): #Since july has id number 48, aug 49 and sept 50
        
        #Clicking on the show 100 entries option
        delay=10
        WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.ID,'tablepress-'+str(month))))
        driver.find_element_by_xpath('//select[@name="tablepress-{}_length"]/option[4]'.format(month)).click()

        #Extracting all the rows of the table
        table=driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr[@role="row"]'.format(month))
        
        #Iterating through each row
        for row in table:
            for i,col in enumerate(row.find_elements_by_xpath('td')): #iterating through each column of the row
                if i==1:
                    date.append(col.text)
                if i==2:
                    startup.append(col.text)
                if i==3:
                    industry.append(col.text)
                if i==4:
                    sub_industry.append(col.text)
                if i==5:
                    location.append(col.text)
                if i==6:
                    investor.append(col.text)
                if i==7:
                    inv_type.append(col.text)
                if i==8:
                    amount.append(col.text)
    
    #Creating dataframe
    data=pd.DataFrame()
    data['Date']=date
    data['Start-Up Name']=startup
    data['Industry']=industry
    data['Sub-Industry']=sub_industry
    data['Location']=location
    data['Investor Name']=investor
    data['Investment Type']=inv_type
    data['Amount']=amount
    return data

In [14]:
df=funding_details('https://trak.in/')
df

,Date,Start-Up Name,Industry,Sub-Industry,Location,Investor Name,Investment Type,Amount
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


### Q7. Write a program to scrap all the available details of best gaming laptops from digit.in. 

In [15]:
def bestlaptops():
    driver=webdriver.Edge(r'msedgedriver.exe')
    driver.get('https://www.digit.in/') #loading url
    driver.maximize_window() #maximizing window
    
    #Clicking on the laptop menu
    driver.find_element_by_xpath('//div[@class="menu"]/ul/li[3]/a').click()
    
    #CLicking on best laptops in india
    driver.find_element_by_xpath('//a[text()="Best Gaming Laptops in India"]').click()
    
    #Extracting url of each laptop
    url=[]
    for i in driver.find_elements_by_xpath('//span[@class="datahreflink spec"]'):
        url.append(i.get_attribute('data-href'))

    #Initializing list to save all the features
    model=[]
    os=[]
    type=[]
    resolution=[]
    dispsize=[]
    wc=[]
    ram=[]
    Ram_type=[]
    weight=[]
    dimension=[]
    processor=[]
    cs=[]
    cores=[]
    gprocessor=[]
    cache=[]
    sd_type=[]
    sd_capacity=[]
    battery=[]
    battery_type=[]
    power=[]

    #Iterating through each url
    for u in url:
        spec=[]
        driver.get(u)
        
        #Flag variables
        m=0
        o=0
        lt=0
        reso=0
        d=0
        w=0
        r=0
        rt=0
        lw=0
        ld=0
        p=0
        c=0
        co=0
        gp=0
        ca=0
        sdt=0
        sdc=0
        bb=0
        bt=0
        ps=0
        
        #Iterating each row in the specs table
        for row in driver.find_elements_by_xpath('//div[@class="remaining-details"]/table/tbody/tr'):
            label=row.find_element_by_xpath('td[1]').text

            #Extracting each feature if label matches and setting flag=1 if found
            if label=='Model Name':
                model.append(row.find_element_by_xpath('td[3]').text)
                m=1
            if label=='Operating System (With Version)':
                os.append(row.find_element_by_xpath('td[3]').text)
                o=1
            if label=='Laptop Type':
                type.append(row.find_element_by_xpath('td[3]').text)
                lt=1
            if label=='Resolution':
                resolution.append(row.find_element_by_xpath('td[3]').text)
                reso=1
            if label=='Display Size (In Inches)':
                dispsize.append(row.find_element_by_xpath('td[3]').text)
                d=1
            if label=='Wireless Connectivity':
                wc.append(row.find_element_by_xpath('td[3]').text)
                w=1
            if label=='Ram Included (In Gb)':
                ram.append(row.find_element_by_xpath('td[3]').text)
                r=1
            if label=='Ram Type':
                Ram_type.append(row.find_element_by_xpath('td[2]').text)
                rt=1
            if label=='Laptop Weight (In Kgs)':
                weight.append(row.find_element_by_xpath('td[3]').text)
                lw=1
            if label=='Laptop Dimension (In Mm)':
                dimension.append(row.find_element_by_xpath('td[3]').text)
                ld=1
            if label=='Processor Model Name':
                processor.append(row.find_element_by_xpath('td[3]').text)
                p=1
            if label=='Clock Speed':
                cs.append(row.find_element_by_xpath('td[3]').text)
                c=1
            if label=='Cores':
                cores.append(row.find_element_by_xpath('td[3]').text)
                co=1
            if label=='Graphics Processor':
                gprocessor.append(row.find_element_by_xpath('td[3]').text)
                gp=1
            if label=='Cache L3':
                cache.append(row.find_element_by_xpath('td[3]').text)
                ca=1
            if label=='Storage Drive Type':
                sd_type.append(row.find_element_by_xpath('td[3]').text)
                sdt=1
            if label=='Storage Drive Capacity':
                sd_capacity.append(row.find_element_by_xpath('td[3]').text)
                sdc=1
            if label=='Battery Backup (In Hours)':
                battery.append(row.find_element_by_xpath('td[3]').text)
                bb=1
            if label=='Battery Type':
                battery_type.append(row.find_element_by_xpath('td[3]').text)
                bt=1
            if label=='Power Supply':
                power.append(row.find_element_by_xpath('td[3]').text)
                ps=1
        
        #Appending '-' if flag of a feature is not set
        if m==0:
            model.append('-')
        if o==0:
            os.append('-')
        if lt==0:
            type.append('-')
        if reso==0:
            resolution.append('-')
        if d==0:
            dispsize.append('-')
        if w==0:
            wc.append('-')
        if r==0:
            ram.append('-')
        if rt==0:
            Ram_type.append('-')
        if lw==0:
            weight.append('-')
        if ld==0:
            dimension.append('-')
        if p==0:
            processor.append('-')
        if c==0:
            cs.append('-')
        if co==0:
            cores.append('-')
        if gp==0:
            gprocessor.append('-')
        if ca==0:
            cache.append('-')
        if sdt==0:
            sd_type.append('-')
        if sdc==0:
            sd_capacity.append('-')
        if bb==0:
            battery.append('-')
        if bt==0:
            battery_type.append('-')
        if ps==0:
            power.append('-')

    data=pd.DataFrame()
    data['Model Name']=model
    data['Laptop Type']=type
    data['Operating System']=os
    data['Resolution']=resolution
    data['Display Size']=dispsize
    data['Wireless Connectivity']=wc
    data['Ram Included']=ram
    data['Ram Type']=Ram_type
    data['Laptop Weight (In Kgs)']=weight
    data['Laptop Dimension (In Mm)']=dimension
    data['Processor Model Name']=processor
    data['Clock Speed']=cs
    data['Cores']=cores
    data['Graphics Processor']=gprocessor
    data['Cache L3']=cache
    data['Storage Drive Type']=sd_type
    data['Storage Drive Capacity']=sd_capacity
    data['Battery Backup (In Hours)']=battery
    data['Battery Backup (In Hours)']=battery_type
    data['Power Supply']=power
    return data

In [16]:
df=bestlaptops()
df

,Model Name,Laptop Type,Operating System,Resolution,Display Size,Wireless Connectivity,Ram Included,Ram Type,Laptop Weight (In Kgs),Laptop Dimension (In Mm),Processor Model Name,Clock Speed,Cores,Graphics Processor,Cache L3,Storage Drive Type,Storage Drive Capacity,Battery Backup (In Hours),Power Supply
0,Area 51M R2,-,Windows 10 Home,1920 x 1080,17.3,-,16 GB,:,4.1,27.65 x 402.6 x 319.14,10th Generation Intel® Core™ i7-10700,2.90 GHz,8,Intel® UHD Graphics 630,16 MB,SSD,1 TB,-,-
1,m15 R3,-,Windows 10 Home,3840 x 2160,15.6,-,16 GB,:,-,-,10th Generation Intel® Core™ i9-10980HK,-,8,-,16MB,SSD,1 TB,-,-
2,ASUS,-,Windows 10 Home,1920 x 1080,15.6,-,16 GB,:,2.30,35.4 x 25.9 x 2.26,AMD Ryzen™ 9 5900HX,3.3 GHz,-,NVIDIA® GeForce RTX™ 3070,16M,SSD,1 TB,Li-ion,240W
3,ROG Zephyrus G14,Gaming,Windows 10 Home,1920 x 1080,14,Yes,16 GB,:,1.65,32.5 x 22.1 x 1.8,AMD 3rd Generation Ryzen 9,3.3 GHz,Quad Core,NVIDIA GeForce RTX 2060,8 MB,SSD,1 TB,Lithium,180W
4,Legion 5i,-,Windows 10 Pro,1920 x 1080,15.6,-,16 GB,:,2.3,363.06 x 259.61 x 23.57,10th Generation Intel® Core™ i5-10300H,2.50 GHz,4,NVIDIA® GeForce® GTX 1650 4GB,8 MB,SSD,1 TB,-,-
5,ROG Zephyrus Duo 15,-,Windows 10,3840 x 1100,15.6,-,4 GB,:,2.4,268.30 x 360.00 x 20.90,Intel Core i7 10th Gen 10875H,-,-,NVIDIA GeForce RTX 2070 Max-Q,-,SSD,512 GB,-,-
6,Aspire 7 Gaming,-,Windows 10 Home,1920 x 1080,15.6,5.1,8 GB,:,2.15,2.29 x 36.3 x 25.4,AMD Ryzen™ 5-5500U hexa-core,-,-,NVIDIA® GeForce® GTX 1650,-,SSD,512 GB,Li-ion,135 W


### Q8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [17]:
def billionaires():
    driver=webdriver.Edge(r'msedgedriver.exe')
    driver.get('https://www.forbes.com/') #loading url
    driver.maximize_window() #maximizing window

    #CLicking Menu button
    driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[1]/button[1]').click()

    #Hovering over to Billinaires, then to world billionaires and the clicking
    action=ActionChains(driver)
    billionaires=driver.find_element_by_xpath('//div[1]/header/nav/div[3]/ul/li[1]')
    world=driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a')
    action.move_to_element(billionaires).move_to_element(world).click().perform()
    time.sleep(3)

    #Initializing lists to store the features
    rank=[]
    name=[]
    worth=[]
    age=[]
    country=[]
    source=[]
    industry=[]

    while True:

        #Extracting each feature
        for i in driver.find_elements_by_xpath('//div[@class="rank"]'):
            rank.append(i.text)
        for i in driver.find_elements_by_xpath('//div[@class="personName"]/div'):
            name.append(i.text)
        for i in driver.find_elements_by_xpath('//div[@class="netWorth"]/div'):
            worth.append(i.text)
        for i in driver.find_elements_by_xpath('//div[@class="age"]/div'):
            age.append(i.text)
        for i in driver.find_elements_by_xpath('//div[@class="countryOfCitizenship"]'):
            country.append(i.text)
        for i in driver.find_elements_by_xpath('//div[@class="source-column"]/div[1]'):
            source.append(i.text)
        for i in driver.find_elements_by_xpath('//div[@class="category"]/div'):
            industry.append(i.text)

        try:
            #Waiting for the next button to appear then clicking on it
            delay=10
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.XPATH,'//*[@id="gatsby-focus-wrapper"]/div/div/div[3]/div[2]/div[2]/div[2]/div[27]/div[7]/div[2]/button[2]')))
            driver.find_element_by_xpath('//*[@id="gatsby-focus-wrapper"]/div/div/div[3]/div[2]/div[2]/div[2]/div[27]/div[7]/div[2]/button[2]').click()
            time.sleep(2)
        except TimeoutException:
            break #If next button is not found that means we are at the end, hence we come out of the loop
        

    #Creating DataFrame
    data=pd.DataFrame()
    data['Rank']=rank
    data['Name']=name
    data['Worth']=worth
    data['Age']=age
    data['Citizenship']=country
    data['Source']=source
    data['Industry']=industry
    return data

In [18]:
df=billionaires()
df 

,Rank,Name,Worth,Age,Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
2750,2674.,Daniel Yong Zhang,$1 B,49,China,e-commerce,Technology
2751,2674.,Zhang Yuqiang,$1 B,65,China,Fiberglass,Manufacturing
2752,2674.,Zhao Meiguang,$1 B,58,China,gold mining,Metals & Mining
2753,2674.,Zhong Naixiong,$1 B,58,China,conglomerate,Diversified


### Q9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [19]:
def utube_comments(no_of_comments):
    driver=webdriver.Edge(r'msedgedriver.exe')
    driver.get('https://www.youtube.com/') #loading url
    driver.maximize_window() #maximizing window

    search=driver.find_element_by_xpath('//input[@id="search"]')
    search.send_keys('all time low')
    search.send_keys(Keys.ENTER)
    time.sleep(2)

    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID,'video-title')))
    driver.find_element_by_xpath('//a[@id="video-title"]').click()

    comment=[]
    upvote=[]
    tim=[]

    f=0
    while f==0:
        try:
            flag=driver.find_element_by_xpath('//ytd-comment-thread-renderer['+str(no_of_comments)+']/ytd-comment-renderer/div[2]/div[2]/ytd-expander/div/yt-formatted-string[2]')
            f=1
            time.sleep(3)
        except NoSuchElementException:
            driver.execute_script("window.scrollBy(0,10000)")

    for i in driver.find_elements_by_xpath('//*[@id="content-text"]'):
        comment.append(i.text)
    for i in driver.find_elements_by_xpath('//*[@id="vote-count-middle"]'):
        upvote.append(i.text)
    for i in driver.find_elements_by_xpath('//div[@id="header-author"]/yt-formatted-string/a'):
        tim.append(i.text)
    
    data=pd.DataFrame()
    data['Comment']=comment
    data['Upvote']=upvote
    data['Time when comment was posted']=tim
    return data

In [20]:
n=input('Enter minimum number of comments to scrape')
df=utube_comments(n)
df

Enter minimum number of comments to scrape500


,Comment,Upvote,Time when comment was posted
0,i'm leaving this comment here so after a month...,4.1K,2 months ago
1,What's more depressing \n1) the song?\n2) or t...,989,2 months ago
2,Fun fact : covid 19 is making old songs popula...,237,2 months ago
3,It's Lockdown 2021. And music is the only one ...,236,1 month ago
4,I was just sitting and I started singing this ...,338,1 month ago
...,...,...,...
495,Mr. Jon Bellion (the legend) participated in p...,249,3 months ago
496,🔥,1,2 months ago
497,2:33 awesome 🤩,,2 days ago
498,Finally i found a song that can boost me when ...,2,2 weeks ago


### Q10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [23]:
def hostelinfo(place):
    driver=webdriver.Edge(r'msedgedriver.exe')
    driver.get('https://www.hostelworld.com/') #loading url
    driver.maximize_window() #maximizing window
    time.sleep(2)

    #Clicking on the search box
    driver.find_element_by_xpath('//input[@id="location-text-input-field"]').click()
    time.sleep(2)
    
    #Typing place provided
    driver.find_element_by_xpath('//input[@id="search-input-field"]').send_keys(place)

    #CLicking on the first predicted result
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="predicted-search-results"]/li[2]')))
    driver.find_element_by_xpath('//*[@id="predicted-search-results"]/li[2]').click()
    
    #Clicking on the search button
    driver.find_element_by_xpath('//button[@id="search-button"]').click()
    time.sleep(5)

    url=[]
    name=[]
    dist=[]
    rat=[]
    rev=[]
    orev=[]
    pprice=[]
    dprice=[]
    fac=[]
    propdesc=[]

    #Extracting info from first page
    
    #Extracting url
    for i in driver.find_elements_by_xpath('//h2[@class="title title-6"]/a'):
        url.append(i.get_attribute('href'))
        
    #Extracting distance from centre
    for i in driver.find_elements_by_xpath('//a[@class="show-on-map"]/span[1]'):
        dist.append(i.text)

    #Extracting prices
    for i in driver.find_elements_by_xpath('//div[@class="prices-col"]/a[1]'):
        #p
        if i.find_element_by_xpath('div[1]/p').text=='Privates From':
            pprice.append(i.find_element_by_xpath('div[1]/div').text)
        else:
            pprice.append('No Privates Available')
        #d
        if i.find_element_by_xpath('div[2]/p').text=='Dorms From':
            dprice.append(i.find_element_by_xpath('div[2]/div').text)
        else:
            dprice.append('No Dorms Available')

    #Extracring info from rest of the pages
    for page in driver.find_elements_by_xpath('//div[@class="pagination-item"]'):
        page.click()
        time.sleep(2)
        
        #Extracting from rest of the pages for each hostel
        for i in driver.find_elements_by_xpath('//h2[@class="title title-6"]/a'):
            url.append(i.get_attribute('href'))

        #Extracting distance from centre
        for i in driver.find_elements_by_xpath('//a[@class="show-on-map"]/span[1]'):
            dist.append(i.text)

        #Extracting prices
        for i in driver.find_elements_by_xpath('//div[@class="prices-col"]/a[1]'):
            #p
            if i.find_element_by_xpath('div[1]/p').text=='Privates From':
                pprice.append(i.find_element_by_xpath('div[1]/div').text)
            else:
                pprice.append('No Privates Available')
            #d
            if i.find_element_by_xpath('div[2]/p').text=='Dorms From':
                dprice.append(i.find_element_by_xpath('div[2]/div').text)
            else:
                dprice.append('No Dorms Available')

    #Extracting info from each url
    for u in url:
        driver.get(u)

        #Extracting name
        name.append(driver.find_element_by_xpath('//div[@class="title-2"]').text)
        #Extracting rating
        try:
            rat.append(driver.find_element_by_xpath('//div[@class="score orange big"]').text)
        except NoSuchElementException:
            rat.append('No Rating')
        #Extracting review
        try:
            rev.append(driver.find_element_by_xpath('//div[@class="reviews"]').text)
        except NoSuchElementException:
            rev.append('0 Total Reviews')
        #Extracting overallrating
        orev.append(driver.find_element_by_xpath('//div[@class="keyword"]/span').text)
        #Extracting facilities
        s=''
        for j in driver.find_elements_by_xpath('//ul[@class="facility-sections"]/li[1]/ul/li'):
            s=s+j.text+'/'
        fac.append(s)
        #Extracting Property Description
        try:
            propdesc.append(driver.find_element_by_xpath('//div[@class="flex-80"]/div[2]/div/div[2]').text)
        except NoSuchElementException:
            propdesc.append(driver.find_element_by_xpath('//div[@class="flex-80"]/div/div/div[2]').text)


    #Creating Dataframe
    data=pd.DataFrame()
    data['Name']=name
    data['Distance from Centre']=dist
    data['Ratings']=rat
    data['Review']=rev
    data['Overall Review']=orev
    data['Private Room Price']=pprice
    data['Dorm Price']=dprice
    data['Facilities']=fac
    data['Property Description']=propdesc
    return data

In [24]:
df=hostelinfo('London')
df

,Name,Distance from Centre,Ratings,Review,Overall Review,Private Room Price,Dorm Price,Facilities,Property Description
0,St Christopher's Village,Hostel - 1.8km from city centre,8.9,10814 Total Reviews,Fabulous,Rs3391,Rs1360,Linen Included/Free City Maps/Free WiFi/Free I...,COVID 19 Policy Update.\nIn response to Corona...
1,Generator London,Hostel - 3km from city centre,7.5,6729 Total Reviews,Very Good,Rs8323,Rs1987,Linen Included/Free City Maps/Free WiFi/Free I...,Generator London is a design hotel-hostel loca...
2,Safestay London Kensington Holland Park,Hostel - 5.9km from city centre,7.9,1070 Total Reviews,Very Good,No Privates Available,Rs1017,Linen Included/Free WiFi/,Safestay Holland Park\n\nSafestay Holland Park...
3,"PubLove @ The Crown, Battersea",Hostel - 4.7km from city centre,7.9,204 Total Reviews,Very Good,No Privates Available,Rs1315,Linen Included/Free City Maps/Free WiFi/,Do London like a local.\nStay just outside the...
4,Leman House,Hostel - 3.6km from city centre,No Rating,0 Total Reviews,No Rating,Rs5857,No Dorms Available,Free WiFi/Free Internet Access/,People come from different parts of the world ...
...,...,...,...,...,...,...,...,...,...
84,The Dover,Hotel - 1.9km from city centre,No Rating,0 Total Reviews,No Rating,Rs7193,No Dorms Available,Free Breakfast/Free City Maps/Towels Included/...,"With friendly staff, excellent service and an ..."
85,Park Hotel Essex,Hotel - 24.1km from city centre,No Rating,0 Total Reviews,No Rating,Rs3597,No Dorms Available,Free Breakfast/Free Parking/,This Hotel is the right choice for visitors wh...
86,Cranbrook Hotel,Hotel - 14.8km from city centre,No Rating,0 Total Reviews,No Rating,Rs3597,No Dorms Available,Free Breakfast/Free Parking/,We are located about twenty minutes by tube fr...
87,St. Athans,Bed and Breakfast - 2.9km from city centre,No Rating,0 Total Reviews,No Rating,Rs3877,No Dorms Available,Linen Included/Free City Maps/Towels Included/...,"The St Athans Hotel is a proudly simple, famil..."
